In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-12 22:41:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.81MB/s    in 0.6s    

2024-03-12 22:41:42 (1.81 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [59]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [60]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [61]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [62]:
chars = sorted(list(set(text))) # 알파벳, 숫자, 특수문자 등등
vocab_size = len(chars)
print(''.join(chars))   
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [63]:
# 입력값을 토큰화 
# 일반적으로 사람들은 sub word encoding을 사용하지만, 단순하게 유지하기 위해 우리가 사용하는 것은 문자 레벨 수준의 토크나이저. 

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)} 

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# 두 개의 정수를 임의의 문자열로 변환하고 다시 변환하는 것과 같음

print(encode('hii there'))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [2]:
# encode된 텍스트를 파이토치 텐서로 변환합니다.
# 아래의 정수 뭉치들은 위에서 input.txt의 [:1000]과 동일한 텍스트를 나타냅니다.

import torch 

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

NameError: name 'encode' is not defined

In [65]:
# 훈련데이터와 검증 데이터를 분할합시다.

n = int(0.9*len(data)) # 90%를 훈련데이터로 사용
train_data = data[:n]
val_data = data[n:] 

In [66]:
# transformer를 훈련할 때 모든 텍스트를 넣으면 비용이 많이 들기때문에, chunk 단위로 덩어리를 나눠서 사용합니다.

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) 이 값들을 우리가 Transformer에 연결할 때, 이것이 말하는 것은 실제로 동시에 훈련하여 현재의 모든 위치에서 예측을 할 것이라는 것입니다.
9개의 문자 덩어리에는 실제로 8개의 개별 예시가 포함되어 있습니다. 

위의 예시에서 47의 맥락에서 보았을 때 18의 다음에 올 가능성이 높고, 18 47 56의 맥락에서 다음으로 57이 올 수 있다는 것을 알 수 있습니다.  

In [67]:
# x는 Transformer에 대한 입력이며 첫 번째 블록 크기 문자가 될 것입니다. 
# y는 다음 블록 크기가 됩니다. 문자이므로 1만큼 오프셋됩니다. 
# 하나의 컨텍스트부터 블록 크기의 컨텍스트까지를 사용하여 여기에 있는 8개의 예제 모두에 대해 훈련합니다.
# Transformer는 하나의 컨텍스트를 사용하여 다음 문자를 예측하고, 그 예측을 사용하여 다음 문자를 예측합니다. 이렇게 하면 블록 크기의 모든 것을 예측할 수 있고 블록 크기 이후에는 자르기를 시작합니다.
# 왜냐하면 Transformer는 다음 문자를 예측할 때 블록 크기 이상의 입력을 받지 않기 때문입니다. 

x = train_data[:block_size]
y = train_data[1:block_size+1]  

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [68]:
torch.manual_seed(1337)

batch_size = 4 # transformer의 모든 순방향 역방향 패스를 처리하는 독립 시퀀스의 수 
block_size = 8 # 예측을 위한 최대 컨텍스트의 길이

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,) ) # 무작위로 선택된 인덱스의 배열 생성 
    # 0~len(data) - block_size 까지의 범위에서 정수 무작위 선택. 
    #(batch_size,)는 함수에게 생성할 무작위 정수의 개수를 알려줍니다.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # 결과적으로, 이 함수는 0부터 len(data) - block_size 범위 내에서 무작위로 batch_size 개수만큼의 정수를 선택
    return x, y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("---------------------------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44

In [69]:
# 간단한 신경망 

import torch 
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337) 

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # vocab_size x vocab_size (어휘의 크기 사용)
        # idx를 전달할 때 입력의 모든 단일 정수가 이 임베딩 테이블을 참조하고 뽑을 것임.
        # 인덱스에 해당하는 임베딩 테이블의 행을 꺼내므로 여기에서는 24를 임베딩 테이블로 이동하여 24번째 행을 뽑고 43번째 행을 차단

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of intergers
        logits = self.token_embedding_table(idx) #(B,T,C) (배치, 텐서, 채널) ex(4,8,65)
        # 여기서 logits은 신경망 모델이 생성한 예측값임. BigramLanguageModel을 통해 얻어진 각 단어에 대응하는 다음 단어의 로짓값을 나타냄.
        # logit이란 확률을 로그 오즈로 변환한 값으로, 신경망이 특정 클래스에 속할 원시 예측 확률을 나타내기 전의 값입니다. 

        if targets is None:
            loss = None
        else:
            # F.cross entropy 손실 함수에 적합한 형태로 만든다.
            B, T , C = logits.shape
            logits = logits.view(B*T, C) # (B*T, C) (32, 65)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits , loss

    def generate(self, idx, max_new_tokens):
        # idx는 (B,T) 사이즈의 정수 텐서입니다.
        for _ in range(max_new_tokens):
            # 예측값을 얻기 위해 마지막 토큰을 사용합니다.
            logits, loss = self(idx)
            # 마지막 타임 스텝에 집중합니다.
            logits = logits[:,-1,:] # (B,C)가 됩니다.
            # 로짓을 소프트맥스로 변환합니다.
            probs = F.softmax(logits, dim=-1) # (B,C)가 됩니다.
            # distribution에서 샘플링합니다.
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # 샘플링된 idx_next를 idx에 추가합니다.
            idx = torch.cat((idx, idx_next), dim=-1)  # (B, T+1)
        
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# idx를 1x1 크기의 텐서로 0으로 초기화되며, 데이터 타입은 정수임. 언어 모델에서는 주로 시작 토큰을 나타내는데 사용한다.
print(decode(m.generate( idx = torch.zeros((1,1), dtype=torch.long) , max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [70]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [71]:
batch_size = 32 # 배치 사이즈 32

for steps in range(10000):

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)    
    optimizer.zero_grad(set_to_none=True)   
    loss.backward()
    optimizer.step()

print(loss.item())

2.572469472885132


In [72]:
print(decode(m.generate( idx = torch.zeros((1,1), dtype=torch.long) , max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercckehathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThineent.

Lavinde.
athave l.
KEONGBUCHandspo be y,-hedarwnoddy scace, tridesar, wne'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers w dere! ABer wotouciullle's


기존의 예측값 : Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3 

학습 후 예측값 (loss 2.5) : 
PENund rron in tenghe;

CROUClereave t omy is.
KISThos, bucesome, l.
ENENRER:
Wiero
DUSe, ysoupad it

### The mathematical trick in self attention

In [73]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch time channel
x = torch.randn(B, T, C) # (4, 8, 2)

x.shape

torch.Size([4, 8, 2])

In [74]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [75]:
# 해당 시퀀스의 모든 토큰에 대해 독립적으로 모든 단일 

xbow = torch.zeros((B,T,C))

for b in range(B):  # 배치 순회 
    for t in range(T): # 시퀀스 내 위치 순회
        xprev = x[b, :t+1] # (t, C)      # 현재 위치 t와 그 이전 위치의 모든 토큰을 선택. 여기서 x[b:t+1]은 배치의 b번째 항목에서 처음부터 t+1까지의 토큰 선택하여 xprev에 저장
        xbow[b,t] = torch.mean(xprev, 0) # 선택된 텐서에 대해 0번째 차원에 대한 평균 계산. 각 채널에 대해 현재 시간 단계까지의 모든 값의 평균을 계산함. 

# 결론 : xbow는 시퀀스의 각 단계에서 이전 단계들의 정보를 종합적으로 반영하는 새로운 방식

In [76]:
# version2
wei = torch.tril(torch.ones(T,T)) 
wei = wei / wei.sum(1, keepdim=True)    
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)    

torch.allclose(xbow, xbow2)

False

In [77]:
xbow[0] 

# 아래의 결과에서 첫번째 행은 그대로임. 
# 두번째는 첫번째 행과 두번째 행의 평균 , 세번째는 첫번째, 두번째, 세번째 행의 평균, 네번째는 첫번째, 두번째, 세번째, 네번째 행의 평균을 나타냄.
# 마지막 행은 모든 토큰의 평균임. 
# 하지만 비효율적임

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [78]:
torch.tril(torch.ones((3,3)))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [79]:
# 효율적인 방법 (행렬)

torch.manual_seed(42)

a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a,1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b 

print('a = ')
print(a)
print('--')
print('b= ')
print(b)
print('--')
print('c = ')
print(c)    

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b= 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [82]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [87]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril ==0 , float('-inf'))
# wei = F.softmax(wei, dim=1)
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [80]:
#version3

tril = torch.tril(torch.ones(T,T)) # 하삼각행렬
wei = torch.zeros((T,T)) # (8,8)
wei = wei.masked_fill(tril ==0 , float('-inf')) # 마스크를 사용하여 하삼각행렬의 0을 -inf로 채움   
wei = F.softmax(wei, dim=1) # (8,8)
xbow3 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

False

In [ ]:
xbow[0] , xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

## self attention

In [97]:
# v4 self-attention
# self-attention은 각 위치의 모든 단일 노드 또는 모든 단일 토큰은 두 개의 벡터를 방출하고 쿼리와 키를 방출합니다.
# 쿼리 벡터는 '내가 찾는 것' 이고 키 벡터는 '내가 포함하는 것' 입니다. 
# 쿼리와 키 벡터는 내적을 통해 유사도를 계산하고 이를 소프트맥스로 변환하여 가중치를 얻습니다.
# 이제 시퀀스에서 이러한 토큰 간의 유사성을 얻는 방법은 기본적으로 점곱(dot product)를 수행하는 것입니다.

# torch.tril은 하삼각행렬로 만들어서 미래의 정보를 차단함. 시퀀스 생성 작업에서 입력 시퀀스의 각 위치가 그 위치와 이후 위치의 정보만 사용하도록 제한하기 위해 사용.

torch.manual_seed(1337)

B,T,C = 4,8,32
x = torch.randn(B,T,C) # (4,8,32)   

head_size = 16
key = nn.Linear(C, head_size, bias=False) # (32, 16)
query = nn.Linear(C, head_size, bias=False) # (32, 16)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T,T)) # 하삼각행렬
# wei = torch.zeros(T,T) # (8,8)
wei = wei.masked_fill(tril ==0 , float('-inf')) # 마스크를 사용하여 하삼각행렬의 0을 -inf로 채움
wei = F.softmax(wei, dim=-1) # (8,8)

v = value(x)
out = wei @ v
# out = wei @ x 

out.shape

torch.Size([4, 8, 16])

In [99]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

### Layer norm

- Layer normalization 은 batch norm과 매우 비슷합니다. 

In [3]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

배치 정규화 (Batch Normalization)
열 단위 정규화: 배치 정규화는 주로 신경망의 각 층에서 활성화 출력의 특성 차원에 대해 작동합니다. 각 특성(또는 채널)에 대한 평균과 분산을 계산할 때, 현재 배치 내의 모든 샘플(데이터 포인트)을 고려합니다. 즉, 같은 특성(열)에 속하는 모든 데이터 포인트들의 평균과 분산을 계산하여 각 특성을 정규화합니다. 이 방식은 특히 컨볼루션 네트워크에서 각 채널별로 적용되며, 데이터의 배치 크기에 의존적입니다.


레이어 정규화 (Layer Normalization)
행 단위 정규화: 반면, 레이어 정규화는 각 데이터 샘플 내에서 작동합니다. 즉, 하나의 데이터 포인트 내의 모든 특성(또는 시간 단계)에 대해 평균과 분산을 계산하여 정규화를 수행합니다. 이는 모든 특성들이 동일한 스케일을 가지도록 만들며, 각 데이터 포인트(행)가 독립적으로 정규화됩니다. 레이어 정규화는 배치 크기와 무관하게 일관된 성능을 제공하며, 특히 시퀀스 데이터를 다루는 RNN이나 Transformer 같은 모델에 유용합니다.

아래의 결과를 보면 Batch norm은 열 단위로 정규화가 진행되고 Layer norm은 행 단위로 정규화가 진행된다.

In [4]:
x[:,0].mean(), x[:,0].std() # mean,ㅁstd of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [5]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-3.5763e-09), tensor(1.0000))

In [6]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5089, val loss 2.5057
step 300: train loss 2.4197, val loss 2.4335
step 400: train loss 2.3502, val loss 2.3564
step 500: train loss 2.2965, val loss 2.3130
step 600: train loss 2.2410, val loss 2.2500
step 700: train loss 2.2059, val loss 2.2189
step 800: train loss 2.1640, val loss 2.1874
step 900: train loss 2.1244, val loss 2.1510
step 1000: train loss 2.1021, val loss 2.1285
step 1100: train loss 2.0704, val loss 2.1193
step 1200: train loss 2.0383, val loss 2.0796
step 1300: train loss 2.0248, val loss 2.0645
step 1400: train loss 1.9927, val loss 2.0370
step 1500: train loss 1.9688, val loss 2.0293
step 1600: train loss 1.9623, val loss 2.0480
step 1700: train loss 1.9411, val loss 2.0145
step 1800: train loss 1.9066, val loss 1.9942
step 1900: train loss 1.9093, val loss 1.9899
step 2000: train loss 1.8826, val loss 1.9942
step 2100: train loss 1.